In [1]:
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization, add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import keras
from NpyDataGenerator import NpyDataGenerator
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import os

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list="0"
sess = tf.Session(config=config)
K.set_session(sess)

In [3]:
model_name = "CNN"
num_classes = 13
classes = ['0gaikan', '1genkan', '2ima', '3kittin', '4huro', '5toire', '6senmen', '7syuno', '8beranda', '9entoransu', '10tyusyajo', '11setubi', '12sonota']
work_dir = '/home/onimem/work/housing-classifier'
input_shape=(224, 224, 3)

In [4]:
def CNN():
    model = Sequential()
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5)))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [5]:
%pylab inline --no-import-all
def plot_history(history, save_name):
    # print(history.history.keys())

    # 損失の履歴をプロット
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['loss', 'val_loss'], loc='lower right')
    plt.savefig(save_name + "/loss.png")
    plt.show()
    
    # 精度の履歴をプロット
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(save_name + "/acc.png")
    plt.show()
    
    # CSV出力
    df = pd.DataFrame({
        'loss' : history.history['loss'],
        'val_loss' : history.history['val_loss'],
        'acc' : history.history['acc'],
        'val_acc' : history.history['val_acc']
    })
    df.to_csv(save_name + '/history.csv')

Populating the interactive namespace from numpy and matplotlib


In [6]:
train_datagen = NpyDataGenerator()
test_datagen = NpyDataGenerator()
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [ ]:
train_dir = pathlib.Path(work_dir + '/data/')
test_dir = pathlib.Path(work_dir + '/data/')    
    
model = CNN()
for layer in model.layers:
    layer.trainable = True
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
    
history = model.fit_generator(
    train_datagen.flow_from_directory(train_dir,classes,32),
    steps_per_epoch=2,
    epochs=300,
    validation_data=test_datagen.flow_from_directory(test_dir,classes,11),
    validation_steps=2,
    #callbacks=[early_stopping]
)

save_name = work_dir + '/history/' + model_name
os.mkdir(save_name)
    
# 学習履歴をプロット
plot_history(history, save_name)
    
model.save(save_name + '/' + model_name  + '.h5')

Epoch 1/300
2/2 [==============================] - 6s 3s/step - loss: 15.0200 - acc: 0.0469 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/300
2/2 [==============================] - 6s 3s/step - loss: 15.8663 - acc: 0.0156 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/300
2/2 [==============================] - 5s 2s/step - loss: 15.6144 - acc: 0.0312 - val_loss: 15.3855 - val_acc: 0.0455
Epoch 4/300
2/2 [==============================] - 4s 2s/step - loss: 15.6144 - acc: 0.0312 - val_loss: 15.3855 - val_acc: 0.0455
Epoch 5/300
2/2 [==============================] - 4s 2s/step - loss: 15.3626 - acc: 0.0469 - val_loss: 14.6528 - val_acc: 0.0909
Epoch 6/300
2/2 [==============================] - 4s 2s/step - loss: 15.8663 - acc: 0.0156 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/300
2/2 [==============================] - 4s 2s/step - loss: 15.8663 - acc: 0.0156 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 8/300
2/2 [==============================] - 4s 2s/step - loss: 15.6

In [ ]:
from PIL import Image
from keras.preprocessing.image import img_to_array, load_img
from IPython.display import display
from IPython.display import Image as Im
import numpy as np

img = img_to_array(load_img('./data/1genkan/P1020453.jpg', target_size=(224,224)))  ### 224*224にリサイズ
# print(type(img)) <type 'numpy.ndarray'>
print(img.shape)
pilImg = Image.fromarray(np.uint8(img))
display(pilImg)

img = Im('./data/1genkan/P1020453.jpg')
display(img)